In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from pathlib import Path

from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from torch_uncertainty.routines import ClassificationRoutine
from torch_uncertainty.utils import TUTrainer
from lightning.pytorch import Trainer
from torch_uncertainty.models import deep_ensembles, mc_dropout
from torch_uncertainty.transforms import RepeatTarget
import torchvision.transforms as T

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm


In [2]:
results_dict = {}

unique_tasks_1 = ['value_los', 'value_icu']
unique_tasks_2 = ['value_hypoglycemia', 'value_hyperkalemia', 'value_hyponatremia', 'value_anemia', 'value_thrombocytopenia']
unique_tasks_3 = ['value_new_hypertension', 'value_new_hyperlipidemia', 'value_new_acutemi']


all_tasks = [unique_tasks_1, unique_tasks_2, unique_tasks_3]
all_tasks_name = ['general_operation_v1', 'lab_test', 'new_diagnose']

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(hidden_size, num_classes)
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.dropout2 = nn.Dropout(p=0.2)
#         self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = F.relu(self.fc3(x))
        return x


def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01 * lr_mult, momentum=0.9)
    return {"optimizer": optimizer}

In [4]:
max_epochs = 50
batch_size = 64

for i in tqdm(range(len(all_tasks))):
# for i in [1]:
    general_task_name = all_tasks_name[i]

    folder_path = f'same_time_data/{general_task_name}'

    train_x_name = os.path.join(folder_path, 'x_train.csv')
    train_y_name = os.path.join(folder_path, 'y_train.csv')
    val_x_name = os.path.join(folder_path, 'x_val.csv')
    val_y_name = os.path.join(folder_path, 'y_val.csv')
    test_x_name = os.path.join(folder_path, 'x_test.csv')
    test_y_name = os.path.join(folder_path, 'y_test.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    X_val = pd.read_csv(val_x_name).to_numpy()
    X_test = pd.read_csv(test_x_name).to_numpy()

    for j in tqdm(range(len(all_tasks[i]))):
    # for j in range(1):
        specific_task_name = all_tasks[i][j]
        y_train = pd.read_csv(train_y_name)[specific_task_name].astype(int).to_numpy()
        y_val = pd.read_csv(val_y_name)[specific_task_name].astype(int).to_numpy()
        y_test = pd.read_csv(test_y_name)[specific_task_name].astype(int).to_numpy()

        assert len(np.unique(y_train)) == 2
        # Create class weights
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weights = torch.tensor(class_weights, dtype=torch.float)

        X_train = torch.tensor(X_train).float()
        X_val = torch.tensor(X_val).float()
        X_test = torch.tensor(X_test).float()

        y_train = torch.tensor(y_train).long()
        y_val = torch.tensor(y_val).long()
        y_test = torch.tensor(y_test).long()

        # Create TensorDatasets
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)

        # Create DataLoaders
        train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        input_size = X_train.shape[1]
        hidden_size = 128
        num_classes = 2
        model = NN(input_size, hidden_size, num_classes)

        mc_model = mc_dropout(model, num_estimators=5, last_layer=False, on_batch=False)

        # The routine is a wrapper of the model that contains the training logic with the metrics, etc
        routine = ClassificationRoutine(
            num_classes=2,
            model=mc_model,
            loss=nn.CrossEntropyLoss(weight=class_weights),
            optim_recipe=optim_recipe(mc_model),
            is_ensemble=True,
        )
        trainer = Trainer(accelerator="gpu", max_epochs=max_epochs, enable_progress_bar=False)

        trainer.fit(model=routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

        # routine.model.on_batch = True
        ens_perf = trainer.test(model=routine, dataloaders=[test_dl])

        results_dict[specific_task_name] = ens_perf
        
        del trainer, routine, model, mc_model

  0%|                                                     | 0/2 [00:00<?, ?it/s]Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type             | Params | Mode 
----------------------------------------------------

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.23323248326778412    │
│       test/cal/aECE       │    0.2332317978143692     │
│       test/cls/Acc        │    0.6681393980979919     │
│      test/cls/Brier       │    0.5245985870263327     │
│     test/cls/Entropy      │    0.2221468687057495     │
│       test/cls/NLL        │    1.3136138916015625     │
│   test/ens_Disagreement   │    0.10171820968389511    │
│     test/ens_Entropy      │    0.1731880009174347     │
│        test/ens_MI        │    0.04895883426070213    │
│       test/sc/AUGRC       │    0.13760481774806976    │
│       test/sc/AURC        │    0.2450123280286789     │
│     test/sc/Cov@5Risk     │            nan            │
│    test/sc/Risk@80Cov     │    0.29631900787353516    │
└───────────────────────────┴───────────────────────────┘


 50%|██████████████████████▌                      | 1/2 [00:12<00:12, 12.18s/it]/tmp/ipykernel_35542/1063529813.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
/tmp/ipykernel_35542/1063529813.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val).float()
/tmp/ipykernel_35542/1063529813.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test).float()
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook envir

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │   0.044648341834545135    │
│       test/cal/aECE       │    0.04170338809490204    │
│       test/cls/Acc        │    0.9464899301528931     │
│      test/cls/Brier       │    0.4904110414657666     │
│     test/cls/Entropy      │   0.041206322610378265    │
│       test/cls/NLL        │    0.4460793435573578     │
│   test/ens_Disagreement   │   0.013942072167992592    │
│     test/ens_Entropy      │    0.0343695767223835     │
│        test/ens_MI        │   0.006836750544607639    │
│       test/sc/AUGRC       │    0.02200920693576336    │
│       test/sc/AURC        │    0.04605722799897194    │
│     test/sc/Cov@5Risk     │    0.9779086709022522     │
│    test/sc/Risk@80Cov     │    0.04171779006719589    │
└───────────────────────────┴───────────────────────────┘


  0%|                                                     | 0/5 [00:00<?, ?it/s]Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      

...


 33%|███████████████                              | 1/3 [00:04<00:08,  4.18s/it]/tmp/ipykernel_35542/1063529813.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
/tmp/ipykernel_35542/1063529813.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val).float()
/tmp/ipykernel_35542/1063529813.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test).float()
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook envir

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.09879419207572937    │
│       test/cal/aECE       │    0.09226758778095245    │
│       test/cls/Acc        │    0.8703071475028992     │
│      test/cls/Brier       │    0.5578735984827355     │
│     test/cls/Entropy      │    0.08967837691307068    │
│       test/cls/NLL        │    0.7446495294570923     │
│   test/ens_Disagreement   │    0.02571103535592556    │
│     test/ens_Entropy      │    0.07867154479026794    │
│        test/ens_MI        │   0.011006835848093033    │
│       test/sc/AUGRC       │   0.048631567507982254    │
│       test/sc/AURC        │    0.09628428518772125    │
│     test/sc/Cov@5Risk     │            nan            │
│    test/sc/Risk@80Cov     │    0.09517045319080353    │
└───────────────────────────┴───────────────────────────┘


 67%|██████████████████████████████               | 2/3 [00:08<00:04,  4.17s/it]/tmp/ipykernel_35542/1063529813.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
/tmp/ipykernel_35542/1063529813.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val).float()
/tmp/ipykernel_35542/1063529813.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test).float()
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook envir

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │     0.028898810967803     │
│       test/cal/aECE       │   0.023418501019477844    │
│       test/cls/Acc        │    0.9647326469421387     │
│      test/cls/Brier       │    0.6347072599531617     │
│     test/cls/Entropy      │   0.046374689787626266    │
│       test/cls/NLL        │    0.21524670720100403    │
│   test/ens_Disagreement   │   0.010693970136344433    │
│     test/ens_Entropy      │    0.04165804013609886    │
│        test/ens_MI        │   0.004716650117188692    │
│       test/sc/AUGRC       │   0.010805790312588215    │
│       test/sc/AURC        │    0.02139848656952381    │
│     test/sc/Cov@5Risk     │            1.0            │
│    test/sc/Risk@80Cov     │   0.019886363297700882    │
└───────────────────────────┴───────────────────────────┘


100%|████████████████████████████████████████████| 3/3 [21:15<00:00, 425.33s/it]


In [5]:
# import json
# with open('results_model_uq_v2/results_single_task_mc_dropout.json', 'w') as f:
#     json.dump(results_dict, f)

import json
with open('results_model_uq_v3/results_single_task_mc_dropout.json', 'w') as f:
    json.dump(results_dict, f)